MLP Langage Des Signes (Classifie si les formes faites avec les mains correspondent au signes de l'alphabet, les testes et entrainnements étant réalisés avec des images)

In [12]:
import cv2
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input
from keras.utils import to_categorical

# Chemins et paramètres
TRAIN_DIR = "Train/"
TEST_DIR = "Test/"
IMG_SIZE = 64
CATEGORIES = ['A', 'B', 'C', 'D', 'E']  # Données limitées jusqu'à E pour éviter la surcharge

# Fonction de chargement
def load_data(directory):
    data = []
    labels = []
    for category in CATEGORIES:
        path = os.path.join(directory, category)
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            data.append(img)
            labels.append(category)
    return np.array(data, dtype='float32') / 255.0, np.array(labels)

# Charger les données d'entraînement
print("Début du chargement des données d'entraînement...")
train_data, train_labels = load_data(TRAIN_DIR)
train_data = train_data.reshape(-1, IMG_SIZE * IMG_SIZE)  # Aplatir pour le MLP

# Charger les données de test
print("Début du chargement des données de test...")
test_data, test_labels = load_data(TEST_DIR)
test_data = test_data.reshape(-1, IMG_SIZE * IMG_SIZE)  # Aplatir pour le MLP

# Encodage des étiquettes
print("Encodage des étiquettes...")
label_encoder = LabelEncoder()
train_labels = to_categorical(label_encoder.fit_transform(train_labels), num_classes=len(CATEGORIES))
test_labels = to_categorical(label_encoder.transform(test_labels), num_classes=len(CATEGORIES))

# Créer le modèle MLP
print("Création du modèle MLP...")
model = Sequential([
    Input(shape=(IMG_SIZE * IMG_SIZE,)),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(len(CATEGORIES), activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
print("\nDébut de l'entraînement du modèle...")
model.fit(train_data, train_labels, epochs=5, batch_size=32)

# Évaluer le modèle
print("\nDébut de l'évaluation sur les données de test...")
loss, accuracy = model.evaluate(test_data, test_labels)
print(f"\nTaux de prédiction réussie (précision) : {accuracy * 100:.2f}%")
print(f"\nPerte sur les données de test : {loss:.4f}")

# Étape 8 : Définition de la fonction de prédiction (adaptée pour MLP)
print("Étape 8 : Définition de la fonction de prédiction...")
def predict_sign(image):
    if image is None:
        print("Impossible de capturer l'image.")
        return None
    img = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    img = img.reshape(1, IMG_SIZE * IMG_SIZE)  # Aplatir pour correspondre à l'entrée du MLP
    prediction = model.predict(img)
    return CATEGORIES[np.argmax(prediction)]

# Étape 9 : Prise d'une seule photo avec la webcam et analyse
print("Étape 9 : Lancement de la prise d'une seule photo avec la webcam...")
cap = cv2.VideoCapture(0)  # 0 est l'index de la webcam par défaut
if not cap.isOpened():
    print("Erreur : Webcam non détectée.")
    exit()

print("Positionnez votre main et appuyez sur 's' pour prendre une photo, 'q' pour quitter sans prendre de photo.")
photo_taken = False
while not photo_taken:
    ret, frame = cap.read()  # Capture une frame
    if not ret:
        print("Erreur : Impossible de capturer la frame.")
        break

    # Afficher la frame en direct
    cv2.imshow('Appuyez sur "s" pour prendre une photo', frame)

    # Attendre une entrée clavier
    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'):  # Prendre une photo
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        predicted_letter = predict_sign(gray_frame)
        if predicted_letter:
            print(f"Lettre prédite : {predicted_letter}")
            cv2.putText(frame, f"Lettre: {predicted_letter}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow('Photo analysée', frame)
            cv2.waitKey(0)  # Attendre une touche pour voir le résultat
        photo_taken = True
    elif key == ord('q'):  # Quitter sans prendre de photo
        print("Aucune photo prise, programme arrêté.")
        break

# Libérer la webcam et fermer les fenêtres
cap.release()
cv2.destroyAllWindows()
print("Étape 9 terminée : Programme arrêté.")

Début du chargement des données d'entraînement...
Début du chargement des données de test...
Encodage des étiquettes...
Création du modèle MLP...

Début de l'entraînement du modèle...
Epoch 1/5
170/170 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - accuracy: 0.5203 - loss: 1.8660
Epoch 2/5
170/170 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - accuracy: 0.9779 - loss: 0.1358
Epoch 3/5
170/170 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - accuracy: 0.9903 - loss: 0.0490
Epoch 4/5
170/170 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - accuracy: 0.9992 - loss: 0.0113
Epoch 5/5
170/170 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - accuracy: 0.9536 - loss: 0.1332

Début de l'évaluation sur les données de test...
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9893 - loss: 0.0356

Taux de prédiction réussie (précision) : 99.12%

Perte sur les données de test : 0.0300
Étape 8 : Définition de la fonction de prédiction...
Étape 9 : Lancement de la prise d'une seule photo avec la webcam...
Positionnez votre main et appuyez sur 's' pour prend